In [1]:
import pandas as pd
import numpy as np
from Abbott_lookup import ab_lookup 
from product_info_JSON import PRODUCT_INFORMATION

# ---------------------------- End of Lookup Table work ---------------------------------

# Create the Abbott product table

Abbot_products_original = pd.read_csv("../Data/Abbott/Abbott_scrape_data.csv",  usecols=PRODUCT_INFORMATION.keys()) #This could be replaced with output from other parser
Abbot_products = Abbot_products_original.copy() #to be used by future analysis
Abbot_products.drop_duplicates(keep='first', inplace=True)
len(Abbot_products)

632

In [2]:
# just used to explore the unique values in the columns so I could make the XML file with all the look up values

import math
import re
from collections.abc import Iterable
vals = Abbot_products['size_or_weight'].str.split(' ') #extract(r'([a-zA-Z]+)')
s = ''
for i in vals:
    if isinstance(i, Iterable):
        s += ' '.join(i)
s = s.split(' ')
s = set(s)


In [3]:
#import each look up list - will be used to clean the data in next cell

import xml.etree.ElementTree as ET
#get all types and all measurements
root = ET.parse('../Data/Abbott/Abbott_product_types_lookup.xml').getroot()
types_list = [i.text for i in root[0]]
types_list
measure_list = [i.text for i in root[1]]
measure_list
bulk_form_list = [i.text for i in root[2]]
print(root['product_format'])

TypeError: element indices must be integers

In [ ]:
#extract value from list that is found in another list
def exists_in_list(l, val):
    for i in val:
        if i in l:
            return i
    return None


a = Abbot_products['size_or_weight']
split_data = list()
#splits by '/'
for i in a:
    s = ' '.join(str(i).split())
    new_tuple = s.split('/')
    split_data.append(new_tuple)
    
 

In [ ]:
    
lhs = list()
rhs = list()

#lhs to have weight metrics, type and weight number
#Rhs will have bulk sales availablility
for i in split_data:
    if len(i) == 2:
        rhs.append(i[1])
        lhs.append(i[0])
    else:
        lhs.append(i[0])
        rhs.append('')

wt_metric = list()
pr_type = list()
wt = list()
bulk_sales_type = list()
bulk_sales_nums = list()

#will create all the same length lists but with their specific values
for i in lhs:
    v = exists_in_list(i, measure_list)
    wt_metric.append(v)
    v = exists_in_list(i, types_list)
    pr_type.append(v)
    d = i.split(' ')
    prod_wt_num = [el for el in d if bool(re.search(r'\d', el))]
    
    if prod_wt_num:
        wt.append(prod_wt_num[0])
    else:
        wt.append('')


for i in rhs:
    v = exists_in_list(i, bulk_form_list)
    bulk_sales_type.append(v)
    d = i.split(' ')
    prod_wt_num = [el for el in d if bool(re.search(r'\d', el))]
    
    if prod_wt_num:
        bulk_sales_nums.append(prod_wt_num[0])
    else:
        bulk_sales_nums.append('')

    


In [ ]:
Abbot_products.head()

In [ ]:

can_numbers_1 = Abbot_products['Case of X'].str.extract('(\d+)')
can_numbers_2 = Abbot_products['Case of Y'].str.extract('(\d+)')
can_numbers = can_numbers_1 + ', ' + can_numbers_2
can_numbers = can_numbers.values.tolist()
#Abbot_products['product_type_counts'] = can_numbers
for i in range(0, len(can_numbers)):
    #print(can_numbers[i][0])
    if isinstance(can_numbers[i][0], str):
        bulk_sales_nums[i] = can_numbers[i][0]
        bulk_sales_type[i] = 'can'
        #print(can_numbers[i])


In [ ]:
Abbot_products['product_format'] = pr_type
Abbot_products['product_measurement_metric'] = wt_metric
Abbot_products['product_measurement_numeric'] = wt
Abbot_products['bulk_sales_type'] = bulk_sales_type
Abbot_products['bulk_sales_numeric'] = bulk_sales_nums


In [ ]:

Abbot_products.head()

In [ ]:
# ------------- Setup dataframe and columns -----------------

del_cols = ['size_or_weight', 'benefits', 'clinical_indications', 'feature_table_rows', 'usage', 'Case of X', 'Case of Y',
           'Flavours', 'size_or_weight', 'availability', 'Sizes', 'footnotes', 'ingredients']

for c in del_cols:
    if c in Abbot_products.columns:
        del Abbot_products[c]
    

In [ ]:
Abbot_products.head()

In [ ]:
Abbot_products['serving_size_1'] = Abbot_products['serving_size_1'].str.replace('Serving Size:', '')
Abbot_products['serving_size_1'] = Abbot_products['serving_size_1'].str.split('(', expand=True)[0]
Abbot_products['serving_size_1'] = Abbot_products['serving_size_1'].str.strip()
df = Abbot_products['serving_size_1'].str.split(' ', expand=True)

Abbot_products['serving_size_metric'] = df[1]
Abbot_products['serving_size_numeric'] = df[0]



In [ ]:
del_cols = list()

for i in range(1, 4):
    del_cols.append('serving_size_' + str(i))
    del_cols.append('nutrient_table_' + str(i))
    del_cols.append('vitamin_table_' + str(i))
    del_cols.append('mineral_table_' + str(i))
    
for c in del_cols:
    if c in Abbot_products.columns:
        del Abbot_products[c]



In [ ]:
Abbot_products.head()

In [ ]:

# Send to CSV
Abbot_products.to_csv("../Data/Abbott/Abbot_products.csv")


In [46]:
#edit all tables, adding extra columns
import os.path


for prod in Abbot_products['item_id'].str.replace('#:', '').tolist():
    #print(Abbot_products[Abbot_products['item_id'] == prod].values)
    p = "../Data/Abbott/Vitamin_tables/vitamin_table_ " + '66441e' + ".csv"
    if os.path.exists(p):
        vit_table = pd.read_csv(p)
        vit_table.to_csv(p, index=False)
        #vit_table.drop(vit_table.columns[[0]],axis=1,inplace=True)
        print(vit_table)
        #vit_table['vitamin'] =  ''
        #vit_table['unit'] = ''
        #print(vit_table['Unnamed: 0'])
        #vit_table[['vitamin', 'unit']] =  vit_table['Unnamed: 0'].astype(str).str.split(',')
        #print(vit_table['Unnamed: 0'].astype(str).str.split(','))
        
        break

TypeError: parser_f() got an unexpected keyword argument 'columns'

In [11]:
#ONCE SCRAPPER FIXED NEED TO RE-SCRAPE FOR TESTING THIS:



        
        #vit_table[cond][['vitamin','unit']] = (vit_table[cond]['Unnamed: 0'].str.split(',', expand=True))
        #d = vit_table.iloc[0].str.split(',')
        #del vit_table['Unnamed: 0']
        #vit_table.to_csv("../Data/Abbott/Vitamin_tables/vitamin_table_" + prod + ".csv")
    '''
    p = "../Data/Abbott/Nutrition_tables/nutrient_table_" + prod + ".csv"
    if os.path.exists(p):
        
        nutr_table = pd.read_csv("../Data/Abbott/Nutrition_tables/nutrient_table_" + prod + ".csv")
        nutr_table

        nutr_table['nutrient'] = len(nutr_table)*''
        nutr_table['metric'] = len(nutr_table)*''

        nutr_table[['nutrient','metric']] = (nutr_table['Unnamed: 0'].str.split(',', expand=True))
        del nutr_table['Unnamed: 0']
        nutr_table.to_csv("../Data/Abbott/Nutrition_tables/vitamin_table_" + prod + ".csv")

    p = "../Data/Abbott/Mineral_tables/mineral_table_" + prod + ".csv"
    if os.path.exists(p):
        min_table = pd.read_csv("../Data/Abbott/Mineral_tables/mineral_table_" + prod + ".csv")
        print(min_table)
        
        min_table['mineral'] = len(min_table)*''
        min_table['metric'] = len(min_table)*''
        
        min_table[['mineral','metric']] = (min_table['Unnamed: 0'].str.split(',', expand=True))
        del min_table['Unnamed: 0']
        min_table.to_csv("../Data/Abbott/Mineral_tables/mineral_table_" + prod + ".csv")
    '''



    Unnamed: 0                        Unnamed: 0.1  Amount per serving  \
0            0                 Vitamin E, IU : 1.5                1.50   
1            1                  Vitamin K, mcg : 8                8.00   
2            2     Thiamin (Vitamin B1), mcg : 100              100.00   
3            3  Riboflavin (Vitamin B2), mcg : 160              160.00   
4            4                Vitamin B6, mcg : 63               63.00   
5            5             Vitamin B12, mcg : 0.26                0.26   
6            6                  Niacin, mcg : 1100             1100.00   
7            7      Folic Acid (Folacin), mcg : 16               16.00   
8            8         Pantothenic Acid, mcg : 470              470.00   
9            9                   Biotin, mcg : 4.6                4.60   
10          10   Vitamin C (Ascorbic Acid), mg : 9                9.00   
11          11                    Choline, mg : 24               24.00   
12          12                  Inosit

In [13]:
# -------------------------------------------

# Set up table which shows all the ingredients for products vertically for analytics
Abbot_products_ingredients_item_type = Abbot_products_original

# Changed from Name to SKU ID "item_type"
Abbot_products_ingredients_item_type = Abbot_products_ingredients_item_type[pd.notnull(Abbot_products_ingredients_item_type['item_id'])]
Abbot_products_ingredients = Abbot_products_ingredients_item_type[['item_id','ingredients']]
Abbot_products_name = Abbot_products_ingredients_item_type['item_id']
Abbot_main_ingredients = pd.concat([Abbot_products_name,
                                    Abbot_products_ingredients['ingredients'].str.split(', ', expand=True)], axis=1)
Abbot_main_ingredients = pd.melt(Abbot_main_ingredients, id_vars = ["item_id"])
Abbot_main_ingredients.dropna(inplace=True)
Abbot_main_ingredients = Abbot_main_ingredients[(Abbot_main_ingredients['value'] != 0)]

Abbot_main_ingredients.rename(columns={'value' : 'ingredient'},inplace=True)
del Abbot_main_ingredients['variable']
Abbot_main_ingredients.head()
Abbot_main_ingredients.sort_values('item_id', inplace=True, ascending=True)
Abbot_main_ingredients.drop_duplicates(keep='first', inplace=True)
Abbot_main_ingredients


# Remove white space and "AND" 
Abbot_main_ingredients.ingredient = Abbot_main_ingredients.ingredient.str.strip()
Abbot_main_ingredients['ingredient'] = Abbot_main_ingredients['ingredient'].str.replace("and ", "", regex=False)
Abbot_main_ingredients['ingredient'] = Abbot_main_ingredients['ingredient'].str.replace(".", "", regex=False)

# Send to CSV
Abbot_main_ingredients.to_csv("../Data/Abbott/ingredients.csv")

In [15]:

# -----------

# Set up table which shows all the flavours for products vertically for analytics
# Changed name to SKU "item_type"

Abbot_product_flavours_item_type = Abbot_products_original
 #This could be replaced with output from other parser
    
Abbot_product_flavours_item_type = Abbot_product_flavours_item_type[pd.notnull(Abbot_product_flavours_item_type['item_id'])]
Abbot_products_name = Abbot_product_flavours_item_type['item_id']
Abbot_product_flavours = Abbot_product_flavours_item_type[['item_id','Flavours']]
Abbot_product_flavours = pd.concat([Abbot_products_name,
                                    Abbot_product_flavours['Flavours'].str.split(', ', expand=True)], axis=1)


Abbot_product_flavours = pd.melt(Abbot_product_flavours, id_vars = ["item_id"])
Abbot_product_flavours['value'] = Abbot_product_flavours['value'].astype(str)

Abbot_product_flavours['value'] = Abbot_product_flavours['value'].str.replace("]", "", regex=False)
Abbot_product_flavours['value'] = Abbot_product_flavours['value'].str.replace("[", "", regex = False)
Abbot_product_flavours['value'] = Abbot_product_flavours['value'].str.replace("'", "", regex=False)
Abbot_product_flavours.dropna(inplace=True)
Abbot_product_flavours = Abbot_product_flavours[(Abbot_product_flavours['value'] != 0)]
del Abbot_product_flavours['variable']
Abbot_product_flavours.rename(columns={'value' : 'Flavours'},inplace=True)

Abbot_product_flavours.sort_values('item_id', inplace=True, ascending=True)
Abbot_product_flavours.drop_duplicates(keep='first', inplace=True)
Abbot_product_flavours

Abbot_product_flavours.to_csv("../Data/Abbott/flavours.csv")
